# MWE 
# Static Vector Multiply by Scalar 
# D. Lahaye

## Importing Libraries

In [1]:
import Gmsh: gmsh 

using LinearAlgebra 
using SparseArrays 
using StaticArrays
using StaticRanges

using BenchmarkTools

using Plots 

## Defining structs 

In [2]:
# struct to hold 2D point
struct Point
    x::Float64   # x coordinates
    y::Float64   # y coordinates 
end
  
  # struct to hold a single mesh element
struct Element
    p1::Point       # coordinates first node 
    p2::Point       # coordinates second node 
    p3::Point       # coordinates third node   
    p4::Point       # coordinates node between first and second node
    p5::Point       # coordinates node between second and third node
    p6::Point       # coordinates node between third and first node

    e1::Int64       # global index first node
    e2::Int64       # global index second node
    e3::Int64       # global index third node
    e4::Int64       # global index fourth node
    e5::Int64       # global index fifth node
    e6::Int64       # global index sixth node
    
    area::Float64   # area of the element
end
  
# struct to hold entire mesh
struct Mesh
    nnodes::Int64               # number of nodes 
    nelems::Int64               # number of elements
    Elements::Array{Element,1}  # list of Elements 
    bndNodeIds::Vector{Int64}   # indices of nodes where Dirichlet bc are applied  
    dofPerElem::Int64           # number of dofs per element 
end

function area_triangle(p1,p2,p3)    
    x12 = p2.x - p1.x; x13 = p3.x-p1.x;
    y12 = p2.y - p1.y; y13 = p3.y-p1.y;
    area_id = x12*y13 - x13*y12; 
    area_id = abs(area_id)/2.
    return area_id 
end

area_triangle (generic function with 1 method)

## Reading Mesh From File

In [3]:
# read elements from mesh file 
function meshFromGmsh(meshFile)    
    
    #Initialize GMSH
    gmsh.initialize()
    
    #Read mesh from file
    gmsh.open(meshFile)

    # Get the mesh nodes
    # Observe that although the mesh is two-dimensional,
    # the z-coordinate that is equal to zero is stored as well.
    # Observe that the coordinates are stored contiguously for computational efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    # sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]

    # Get the mesh elements
    # Observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelems = length(element_ids[1])
      
    # Construct uninitialized array of length nelements  
    Elements = Array{Element}(undef,nelems)

    # Construct the array of elements 
    for element_id in 1:nelems
        e1 = element_connectivity[1][6*(element_id-1)+1]
        e2 = element_connectivity[1][6*(element_id-1)+2]
        e3 = element_connectivity[1][6*(element_id-1)+3]
        e4 = element_connectivity[1][6*(element_id-1)+4]
        e5 = element_connectivity[1][6*(element_id-1)+5]
        e6 = element_connectivity[1][6*(element_id-1)+6]

        p1 = Point(sorted[e1,2], sorted[e1,3])
        p2 = Point(sorted[e2,2], sorted[e2,3])
        p3 = Point(sorted[e3,2], sorted[e3,3])
        p4 = Point(sorted[e4,2], sorted[e4,3])
        p5 = Point(sorted[e5,2], sorted[e5,3])
        p6 = Point(sorted[e6,2], sorted[e6,3])
        
        area = area_triangle(p1,p2,p3);

        Elements[element_id] = Element(p1,p2,p3,p4,p5,p6,e1,e2,e3,e4,e5,e6,area)
    end

    # retrieve boundary nodes by loop over corner point and boundary edges
    node_ids1=[]; node_ids2=[]; node_ids3=[]; node_ids4=[]; 
    node_ids5=[]; node_ids6=[]; node_ids7=[]; node_ids8=[]; 
    node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
    node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
    node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
    node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
    node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,1)
    node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,2)
    node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,3)
    node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,4)
    bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)
    
    # Set DOF per element
    dofPerElement = 36
    
    # Store data inside mesh struct  
    mesh = Mesh(nnodes,nelems,Elements,bnd_node_ids,dofPerElement) 
    
    # Finalize gmsh
    gmsh.finalize()
    
    return mesh 
end

meshFromGmsh (generic function with 1 method)

In [4]:
# read coarse mesh  
mesh_coarse = meshFromGmsh("square-2nd-coarse.msh")
typeof(mesh_coarse)

Info    : Reading 'square-2nd-coarse.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square-2nd-coarse.msh'


Mesh

In [5]:
# read normal mesh
mesh_normal = meshFromGmsh("square-2nd-normal.msh")
typeof(mesh_normal)

Info    : Reading 'square-2nd-normal.msh'...
Info    : 9 entities
Info    : 37 nodes
Info    : 26 elements
Info    : Done reading 'square-2nd-normal.msh'


Mesh

In [6]:
# read fine mesh 
mesh_fine = meshFromGmsh("square-2nd-fine.msh")
typeof(mesh_fine)

Info    : Reading 'square-2nd-fine.msh'...
Info    : 9 entities
Info    : 1969 nodes
Info    : 1028 elements
Info    : Done reading 'square-2nd-fine.msh'


Mesh

## Finite Element Matrix Assembly

In [7]:
# Function to assemble the stiffness matrix
function genLocStiffMat(element::Element)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4; p5 = element.p5; p6 = element.p6;
    e1 = element.e1; e2 = element.e2; e3 = element.e3; e4 = element.e4; e5 = element.e5; e6 = element.e6;
    area = element.area  
    
    Iloc = SVector(e1, e1, e1, e1, e1, e1, e2, e2, e2, e2, e2, e2, e3, e3, e3, e3, e3, e3, e4, e4, e4, e4, e4, e4, e5, e5, e5, e5, e5, e5, e6, e6, e6, e6, e6, e6)

    Jloc = SVector(e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6)
    
    Aloc = SVector(1., -1/6, 1/6, 0., 1/6, -1/6,
                   -1/6, 1/6, -1/6, -1/12, 0., 0.,
                    1/6, -1/6, 1/2, 0., 0., 0.,
                    0., -1/12, 0., 1/6, 0., -1/12,
                    1/6, 0., 0., 0., 1/2, -1/6,
                    -1/6, 0., 0., -1/12, -1/6, 1/6)
    
    Aloc = SVector{36}(Aloc./area)   
            
    return Iloc, Jloc, Aloc 
end

function genStiffMat(mesh::Mesh)

    nelems = mesh.nelems
    dofPerElem = mesh.dofPerElem

    Avalues = zeros(Float64, dofPerElem*nelems)
    I = zeros(Int64, length(Avalues))
    J = zeros(Int64, length(Avalues))

    for i = 1:nelems
        element = mesh.Elements[i]
        Iloc, Jloc, Aloc = genLocStiffMat(element)
        irange = mrange(dofPerElem*i-35, dofPerElem*i)
        I[irange] = Iloc
        J[irange] = Jloc
        Avalues[irange] = Aloc  
    end

    A = sparse(I,J,Avalues)

    return A;
end

genStiffMat (generic function with 1 method)

In [8]:
#mesh = meshFromGmsh("square-2nd.msh")
A = genStiffMat(mesh_fine);
display(A)

1969×1969 SparseMatrixCSC{Float64, Int64} with 22033 stored entries:
⡿⣯⡉⢹⠆⠀⠀⠀⣷⣤⢀⣿⠞⠩⣬⠤⢀⣲⣸⡰⢶⡆⠖⠆⠀⠃⠘⣤⢠⢀⠀⠀⢠⠃⡀⢀⣀⡿⣼⡅
⣇⣈⣿⣿⠅⠀⠀⢀⣿⣾⢈⣻⣀⣒⣶⣖⣿⣿⣯⣻⣃⣷⣇⣀⠀⡀⢀⣰⢰⢸⣱⢈⣁⡀⣇⣈⣋⣘⣺⣱
⠈⠁⠁⠁⣿⣿⣿⡁⡭⣿⣯⢌⠀⢄⠈⢀⢉⡾⢞⣿⣧⣷⣿⣷⣷⣶⣿⣿⣿⣿⣿⣷⣿⣶⣯⣍⡉⠄⢈⢀
⠀⠀⠀⢀⠟⠻⣿⣿⣿⣿⣿⣿⣷⣝⣇⣟⣯⣬⠻⡮⣳⣿⢿⣿⡿⢿⣿⢿⣿⡿⢿⡟⣿⣿⣿⣿⣶⡿⣝⣏
⠙⣿⣻⣿⣧⣯⣿⣿⣿⣿⣿⣿⣏⣵⣿⡻⣻⣿⣯⡿⣿⢿⣾⣽⣭⣤⣬⣀⣼⣅⣍⣡⡕⣤⣼⣭⣟⠽⣿⡿
⣤⣴⣦⣰⡋⢟⣿⣿⣿⣿⣿⣿⣿⣷⣦⣮⡽⢫⣼⢿⣿⣗⡿⢿⢫⠽⣾⢝⢟⠛⢛⣟⣫⠿⣻⡿⣷⣯⣧⣥
⡞⡁⢠⢸⠀⢄⣝⢿⢏⣽⢿⣿⡿⣯⣏⣍⠁⠩⠄⣀⡅⠊⠈⠑⠰⠆⢦⣬⢈⠀⠀⣀⠛⠗⠩⢇⢈⣿⣉⠯
⠂⡟⢸⢿⠂⢀⣭⢽⣿⡻⡨⣿⡏⢽⣿⣿⡧⢬⡪⢊⢢⡵⢀⠈⠀⢀⢄⠰⠘⠀⠨⢠⢀⠭⡔⠉⢥⢨⣿⣧
⢠⣰⣿⣿⣣⡴⡋⣿⣿⣾⡷⣋⡅⡀⡉⣏⣿⣿⣶⣯⣧⡓⣫⠙⠴⢂⡄⠂⡐⠰⡌⠈⠂⠒⠌⠓⠪⡀⢉⣨
⢒⡺⣯⣻⣾⣵⡻⡦⣯⡿⣶⣟⠀⢡⡪⢊⡼⣿⣿⣿⣧⣕⣷⠝⣞⢒⢴⡊⠴⠅⡄⠨⡅⢄⢠⢶⠦⠚⡉⠉
⠸⠷⢭⣼⢭⣿⣽⣾⣿⣟⢿⢿⡡⠉⢌⡶⢭⠻⢍⢿⣿⣿⣟⣴⠛⡠⠅⠁⢠⡡⢄⠦⠢⠅⠎⢬⣤⠵⡫⢪
⠸⠅⠉⢹⢿⣿⣿⣷⣞⣿⣿⣏⢆⠀⡀⠐⣏⠚⣝⠟⢛⣽⣿⣿⣿⣎⡜⢶⡷⡌⠑⡲⡄⠬⡃⠕⣈⡎⡊⠐
⠤⠀⠀⠠⢹⣿⣿⣏⠃⣿⣏⡖⠰⠆⠀⢀⠰⢃⢺⢙⠛⡠⡻⢿⣿⣿⣏⣯⣱⠊⣿⣂⢇⣋⢀⢁⡁⣆⠐⠐
⠒⣤⢀⣰⣿⣿⣿⣟⠂⢻⣞⢟⡈⣷⢀⡑⠠⠉⡰⠳⠅⠁⢲⣍⡯⣽⢟⣵⣿⣿⣳⣯⡦⢔⠖⠉⡈⠇⢁⡃
⠀⢒⣐⣒⣿⣿⣿⡿⠖⢿⣿⠑⠂⠐⠒⠀⢐⡈⠔⠇⠄⡲⡙⠯⡱⠚⣿⣿⣿⣿⣿⣞⣷⡳⡷⢏⠀⠀⠑⠀
⠀⠀⡑⢚⢿⣿⣿⠷⠇⣹⣿⢴⠀⢠⠂⣂⡂⠉⡀⡉⠠⡕⢱⡠⠻⢻⡽⣾⣻⢿⡿⣯⣷⣳⣢⡸⡷⠠⢒⠄
⠤⠒⠁⠸⢻⣿⣿⣿⠑⣭⣯⡞⢿⠄⡄⡔⢨⠀⠁⢍⠌⠆⡀⡍⡭⢱⢈⢏⢽⡻⢽⣻⣿⣿⣿⣍⠔⣆⠚⠂
⠀⢈⡉⢹⡏⢿⣿⣿⡖⣿⣿⡾⠧⢆⡔⠉⢦⠁⢠⣖⡊⣅⢍⠌⠄⢐⡜⠁⡽⢏⣈⡺⡟⢿⣿⣿⣿⠕⡏⠏
⣤⡼⣋⢸⠃⠌⣼⡿⣟⡝⡽⣿⣦⣴⡁⣓⠊⠢⣨⠃⢄⡟⡢⠼⠡⢬⠦⠌⠀⠀⠙⡋⠰⢥⢟⠟⢛⣴⣟⣲
⠖⠿⢞⣺⠂⢐⡷⢽⣿⡿⠍⣿⡧⡜⠿⣿⡃⣰⡇⠈⡫⣊⢊⠈⢐⠀⠥⠰⠑⠀⠘⠔⠺⠀⡯⠍⢻⣹⣿⣿

In [9]:
@time genStiffMat(mesh_coarse);
@time genStiffMat(mesh_normal);
@time genStiffMat(mesh_fine);

  0.000015 seconds (13 allocations: 8.516 KiB)
  0.000014 seconds (13 allocations: 27.312 KiB)
  0.000407 seconds (20 allocations: 1.679 MiB)
